O objetivo dessa projeto é criar uma API que extrai dados de nóticias através da News API [https://newsapi.org/](https://newsapi.org/), e a partir disso transformar esses dados para consumo final.

Esse projeto é composto pelos seguintes arquivos:

- **extract_data.ipynb** - presente arquivo que contém todos os códigos reunidos.
- **app.py** - script com Flask API que gera o ambiente para consumo dos dados.
- **data/**
  - **api_extract.csv** - arquivo com as notícias extraídas da News API;
  - **articles_relevance.csv** - arquivo com relevância dos artigos;
  - **keyword_counts.csv** - arquivo com a quantidade de vezes que cada palavra-chave aparece;
  - **news_data.csv** - arquivo de dados de notícias;
  - **news_per_date.csv** - arquivo com a quantidade de notícias por ano, mês e dia de publicação;
  - **news_per_source_author.csv** - arquivo com a quantidade de notícias por fonte e autor.

Para a criação desse trabalho foi preciso desenvolver alguns critérios para a extração. Onde foram escolhidas três palavras-chave para filtrar as notícias, sendo elas **"DNA sequencing", "gene therapy" e "genetic diseases"**. E com o script de extração feito, as notícias extraídas são armazenadas em formato raw em um csv, para posteriormente ocorrer uma transformação dos dados e consequentemente a disponibilidade para consumo.

A extração ocorre a cada 1 hora onde apenas notícias novas são adicionadas ao arquivo e, se alguma notícia já estiver armazenada, ela não é adicionada novamente, evitando duplicação.

## Extração da News API

In [2]:
import requests
import pandas as pd
from pandas import json_normalize
import time


def fetch_news(api_key, topic):
    url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={api_key}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        articles = data['articles']
        return articles
    else:
        print("Erro ao fazer a solicitação:", response.status_code)
        return None


def update_news():
    api_key = "187be6640acc439db24e18a27eb1ad87"
    topics = ["DNA sequencing", "gene therapy", "genetic diseases"]  # Define palavras-chave

    all_articles = []
    
    for topic in topics:
        articles = fetch_news(api_key, topic)
        if articles:
            all_articles.extend(articles)

    if all_articles:
        df = json_normalize(all_articles)
        # Adiciona uma coluna de id e reorganiza as colunas
        df['id'] = range(1, len(df) + 1)
        cols = df.columns.tolist()
        cols = ['id'] + [col for col in cols if col != 'id']
        df = df[cols]
        
        df.to_csv("data/api_extract.csv", index=False)
        print("Notícias atualizadas com sucesso.")
    else:
        print("Não há novas notícias para adicionar.")


def main():
    update_news()

    while True:
        time.sleep(3600) # Atualização a cada 3600 segundos (=1 hora)
        update_news()


if __name__ == "__main__":
    main()


Notícias atualizadas com sucesso.
Notícias atualizadas com sucesso.
Notícias atualizadas com sucesso.
Notícias atualizadas com sucesso.


KeyboardInterrupt: 

### Dados transformados para consulta do público final

Para essa parte do projeto, são criados novos arquivos csv com os dados extraídos da API de notícias que são transformados, para que assim, possam ser consumidos na API final.

In [3]:
# Quantidade de notícias por ano, mês e dia de publicação
import pandas as pd

df = pd.read_csv("data/api_extract.csv")

df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df['year'] = df['publishedAt'].dt.year
df['month'] = df['publishedAt'].dt.month
df['day'] = df['publishedAt'].dt.day
news_per_date = df.groupby(['year', 'month', 'day']).size().reset_index(name='news_count')


news_per_date.to_csv("data/news_per_date.csv", index=False)

In [4]:
# Quantidade de notícias por fonte e autor
news_per_source_author = df.groupby(['source.name', 'author']).size().reset_index(name='news_count')

news_per_source_author.to_csv("data/news_per_source_author.csv", index=False)

In [5]:
# Quantidade de aparições das 3 palavras-chave por ano, mês e dia de publicação definidas no item 2
keywords = ['DNA sequencing', 'gene therapy', 'genetic diseases']

def contains_keyword(title):
    return any(keyword in title.lower() for keyword in keywords)

for keyword in keywords:
    df[keyword] = df['title'].apply(lambda x: 1 if contains_keyword(x) else 0)

keyword_counts = df.groupby(['year', 'month', 'day'])[keywords].sum().reset_index()

keyword_counts.to_csv("data/keyword_counts.csv", index=False)

## Flask API
Após a geração de todos os dados, é preciso criar uma API para que o usuário final possa consultar tudo que foi gerado. Para isso, utilizamos Flask para disponibilizar as informações, onde cada página feita retorna uma função com o que foi pedido. Para está API foi feito um arquivo adicional chamado "**index.html**" dentro da página templates/ que contém um script básico feito para o front-end da API que facilita o direcionamento entre as páginas.

In [ ]:
from flask import Flask, jsonify, request, render_template
import pandas as pd
import random

app = Flask(__name__)

df_news = pd.read_csv("data/api_extract.csv")
df_news_per_source_author = pd.read_csv("data/news_per_source_author.csv")
df_keyword_counts = pd.read_csv("data/keyword_counts.csv")


# Home
@app.route("/", methods=["GET"])
def homepage():
    return render_template('index.html')


# Retornar todas as notícias
@app.route("/api/news", methods=["GET"])
def get_all_news():
    return jsonify(df_news.to_dict(orient='records'))


# Retornar notícia por id
@app.route('/api/news/<int:id>', methods=['GET'])
def get_news_by_id(id):
    news = df_news.loc[df_news['id'] == id]
    if len(news) == 0:
        return "Notícia não encontrada", 404
    return jsonify(news.to_dict(orient='records')[0])


# Receber outras notícias que não foram mapeadas pela API original
@app.route('/api/news', methods=['POST'])
def add_new_news():
    global df_news
    new_news = request.json  
    df_news = pd.concat([df_news, pd.DataFrame([new_news])], ignore_index=True) 
    df_news.to_csv("data/api_extract.csv", index=False) 
    return jsonify({'message': 'Notícia adicionada com sucesso'}), 201


# Retornar a quantidade de notícias por ano, mês e dia de publicação do item 4.1
@app.route('/api/news/count_by_date', methods=['GET'])
def get_news_count_by_date():
    news_count_by_date = df_news.groupby(['publishedAt']).size().reset_index(name='news_count')
    return jsonify(news_count_by_date.to_dict(orient='records'))


# Retornar a quantidade de notícias por fonte e autor do item 4.2
@app.route('/api/news/count_by_source_author', methods=['GET'])
def get_news_count_by_source_author():
    return jsonify(df_news_per_source_author.to_dict(orient='records'))


# Retornar a quantidade de aparições das 3 palavras-chave por ano, mês e dia de publicação  do item 4.3
@app.route('/api/news/count_by_keyword', methods=['GET'])
def get_news_count_by_keyword():
    return jsonify(df_keyword_counts.to_dict(orient='records'))


if __name__ == '__main__':
    app.run(debug=True)


### Exemplo de como adicionar novas notícias

In [8]:
# Receber outras notícias que não foram mapeadas pela API original
import requests

url = 'http://127.0.0.1:5000//api/news'

# Dados da nova notícia em formato JSON
new_news = {
    "source": {"name": "Projeto Modulo 4"},
    "author": "Grupo 4",
    "title": "Novo artigo adicionado",
    "description": "Exemplo de notícia",
    "url": "https://example.com/article",
    "publishedAt": "2024-04-07T12:00:00Z",
    "content": "Content of the new article"
}

response = requests.post(url, json=new_news)

<p align="center">
  <img src="images/code_results.gif" alt="Alt Text">
</p>